#### Input

In [1]:
# ! pip install pdfplumber
import pdfplumber
import pandas as pd
import numpy as np
import logging
logging.basicConfig(level=logging.INFO)
logging.info("..")
from IPython.display import display

#**
file= r"tbd0.pdf"
sumpage = [0] #pdf-1
volinj= 5
injrep_cal=[6,6,6,6,6,6] #explicit'70','85','100','115','130','qcs' 
samrep= 6
bracrep= samrep


#** 
analyte= 'Arg', 'Glu', 'Lys', 'Pro',  
compage=  [5,6,7,8,9,10,11,12]  
'''
if amin/nonval use neg so calc is valid later. same logic for lbclaim.
'''
Std_conc, QCS_conc= [1005.2352, 302.5836, 311.1600, 50.6400],[1008.0914, 313.8000, 304.2155, 51.4575]   
Volinit_std, Volinit_qcs=[3.5,4.25,5,5.75,6.5], 5
Volser_std, Volser_qcs=[1]*5, 1
Volfin_std, Volfin_qcs=[100]*5, 100
'''
if having other type of unknown which result is needed, 
add them here or change their naming to smt numeric
might need to change their filter in the sections Presum and Sum
'''
namechange= {
    '70% old': r'990',
    '85% old': r'991',
    '100% old': r'992',
    '115% old': r'993',
    '130% old': r'994',
}
samcon= ['1192526 GLA','1192526 P','1192235']
fillvol= [1]*len(samcon) 
lbclaim= [[50,15,15,-1], 
          [-15,-15,-50, 25], 
          [-15, 25,-50, -25]] 

header= ["IS","Roc","Ace"]
sam_0=pd.DataFrame(pd.concat([pd.Series(samcon), pd.Series(fillvol), pd.Series(lbclaim)],axis=1)) #maintain order
sam_0.columns=['Name','Fill','Claim']
sam_0['Claim0']= sam_0['Claim'].str[0]
sam_0['Claim1']= sam_0['Claim'].str[1]
sam_0['Claim2']= sam_0['Claim'].str[2]
sam_0['Claim3']= sam_0['Claim'].str[3]


#**
calib_page =[1,2,4,3] 
calib_page1 =[1,2,4,3]

gh= [[analyte],'Amount','Area']
word_h= ["Lin,",'Slope',"Square","IS",'Amount',"Acetyl"]
word_v= ['Area'] 
reso= [2,2.5,3,3.5] 

INFO:root:..


#### Overview

In [2]:
with pdfplumber.open(file) as pdf:
    dfo=[]
    for i in sumpage:
        page = pdf.pages[i].extract_table()
        dfo.extend(
            [
            [row[i] for i in [i for i, col in enumerate(list(zip(*page))) if col.count(None) < len(page)/2]]
            for row in page
        ]
        )
    dfo=pd.DataFrame(dfo)

dfo.iloc[30:36]

,0,1,2,3,4,5,6,7,8
30,18,1,Calibration Standard,70%-6,AMI-2137,R:C1,5.00,1.0000,Finished
31,19,2,Calibration Standard,85%,AMI-2137,R:C2,5.00,1.0000,Finished
32,20,2,Calibration Standard,85%,AMI-2137,R:C2,5.00,1.0000,Finished
33,21,2,Calibration Standard,85%,AMI-2137,R:C2,5.00,1.0000,Finished
34,22,2,Calibration Standard,85%,AMI-2137,R:C2,5.00,1.0000,Finished
35,23,2,Calibration Standard,85%,AMI-2137,R:C2,5.00,1.0000,Finished


In [3]:
sum_i= dfo[dfo.astype(str).apply(lambda row:row.str.contains('injection name', case=False,na=False),axis=1).any(axis=1)].index[0]
dfo=dfo.iloc[sum_i:]
dfo.columns=dfo.iloc[0,:]
dfo=dfo.iloc[1:]
dfo_0= dfo.copy()

In [ ]:
# Removing rows containg discard/rinse
dfo1_0=dfo_0[dfo_0['Position'].astype(str).str.contains(r'\d+$', regex= True)].copy()
for k,v in namechange.items():
    dfo1_0['Injection Name']= dfo1_0['Injection Name'].str.replace(k,v,regex=False)
dfo1_0=dfo1_0[~dfo1_0.astype(str).apply(lambda row: row.str.contains('Discard|rinse|old', case=False, na=False)).any(axis=1)] ####
dfo1dc_0=dfo1_0[dfo1_0.astype(str).apply(lambda row: row.str.contains('discard|rinse', case=False, na=False)).any(axis=1)]

# Convert columns with numerical values to numeric
dfo1_0["Volume"]=pd.to_numeric(dfo1_0['Volume'], errors='coerce')
dfo1_0['Dilution']=pd.to_numeric(dfo1_0['Dilution'], errors='coerce')
dfo1_0['Level']=pd.to_numeric(dfo1_0['Level'], errors='coerce')

# Group all by rows by volume
vol=dfo1_0.groupby("Volume")['Volume'].mean()
'''
If the mean volume is not unique, or the volume is not the same as indicated, output error
'''
if len(vol)!= 1 or vol.iloc[0]!= volinj:
    print(f'Wrong inj vol\n')
else:
    pass

# Using filter/mask to select rows that are not finished
dfo2=dfo1_0[dfo1_0["Status"] != "Finished"]
if dfo2.shape[0]!=0:
    print(f'not finished\n')
else:
    pass

# Using filter/mask to count the number of each level in the system suitability
calo=[]
for i in ['70%','85%','100%','115%','130%','qcs|check']:
    cal1= (~dfo1_0['Injection Name'].str.contains('bracket', case=False) & dfo1_0['Injection Name'].str.contains(i, case=False)).sum()
    calo.append(cal1)
'''
If the number of each replicate doesn't match with what specified, output error
'''
if np.array_equal(np.array(calo),injrep_cal) != True: 
    print(f'Ck calib replicate\n-------\n {np.array(calo)}\n')
else:
    pass

# Cleaning the names and group them to show the dilution factor
samo= dfo1_0[dfo1_0['Injection Name'].str.contains(r'\b\d{7}\b', regex=True)].copy()
samo['Name']= samo['Injection Name'].str.split('-', n=1).str[0]
samo1=samo.groupby('Name')['Dilution'].describe().iloc[:,:3]

notsamo= dfo1_0[~dfo1_0['Injection Name'].str.contains(r'\b\d{7}\b', regex=True)].copy()
notsamo['Injection Name']= notsamo['Injection Name'].str.split('-', n=1).str[0]
notsamo1= notsamo.groupby('Injection Name')['Dilution'].describe().iloc[:,:3]


dfo1_0['Name']= dfo1_0['Injection Name'].str.split('-', n=1).str[0]

# Using filter/mask to count the number of each bracket in the system suitability
braco1= dfo1_0[dfo1_0['Name'].str.contains('bracket', case=False)]
braco=braco1.groupby('Name')['Name'].describe().iloc[:,3]
'''
If the number of each replicate doesn't match with what specified, output error
'''
if np.array_equal(np.array(braco), np.array([bracrep]*len(np.array(braco)))) != True: ###
    print(f'Ck bracket replicate\n')
else:
    pass

# Group by name to show dilution factor
dfo1d= dfo1_0.groupby('Name')['Dilution'].describe().iloc[:,:3].reset_index()
print(f'DF of all\n-------\n {dfo1d[['Name','mean']]}\n') # count 6/2
'''
Using standard deviation to check if the dilution factor are all same for one entity
'''
if (dfo1d[dfo1d['std'] > 0.0000001]).shape[0] !=0:
    print(f'Check DF of\n{dfo1d[dfo1d['std'] > 0.00001]['Name']}\n')
else:
    pass

# Group by name to show level assignmnent
dfo1l= dfo1_0.groupby('Name')['Level'].describe().iloc[:,:3].reset_index()
'''
Using standard deviation to check if the levels are all same for one entity
'''
if (dfo1l[dfo1l['std'] > 0.0000001]).shape[0] !=0:
    print(f'CHECK level assignment of\n{dfo1l[dfo1l['std'] > 0.00001]['Name']}\n')
else:
    pass
print(f'Level assignment\n-------\n{dfo1l[['Name','mean']].sort_values(by='mean')}\n')


# Group by name to show the position of each entity
dfo1v= dfo1_0.groupby('Name')['Position'].describe().reset_index() #.iloc[:,:3]
'''
Using count and uniqueness to check if the position is unique to each entity
'''
mask= (dfo1v['freq'] != samrep) | (dfo1v['unique'] != 1) 
if dfo1v[mask].any().any():
    print(f'Check vial pos of\n-------\n{(dfo1v[mask])['Name']}')
    print(dfo1v)
else:
    pass

dfo1vv= dfo1_0.groupby('Position')['Name'].describe().reset_index()
if dfo1vv[dfo1vv['unique'] != 1].any().any():
    print(f'\nCHECK vial at\n-------\n{(dfo1vv[dfo1vv['unique'] != 1])['Position']}')
    print(dfo1vv)
else:
    pass


# Filter the positions of samples and verify that they are not injected outside of the sst brackets
samvo= dfo1v[dfo1v['Name'].str.contains(r'\b\d{7}\b', regex=True)]['top']
dfo1dc_0['check']= dfo1dc_0['Position'].str.contains(pd.Series(samvo).iloc[0])
samvocheck= dfo1dc_0[dfo1dc_0['check']==True]
if samvocheck.shape[0]!=0:
    print(f'Sample in discard!\n')
else:
    pass

# Select standards and select level assignments to verify that there are no level assignment in random places
std= dfo1_0[dfo1_0['Type'].str.contains('Standard')]  #if df1_0['Type] exist
dfo1_0['Level']=dfo1_0['Level'].replace(r'^\s*$',np.nan, regex=True).fillna(0)
lev= dfo1_0[dfo1_0['Level']!=0]
lev
if std.equals(lev)!=True:
    print(f'Non-std has level')
else:
    pass

DF of all
-------
                   Name     mean
0                    0   1.0000
1                   00   1.0000
2   100% Final Bracket   1.0000
3           100% SSuit   1.0000
4                 115%   1.0000
5              1192253   1.6666
6          1192526 GLA   1.0000
7            1192526 P  10.0000
8                 130%   1.0000
9                  70%   1.0000
10                 85%   1.0000
11                 QCS   1.0000

Level assignment
-------
                  Name  mean
9                  70%   1.0
10                 85%   2.0
3           100% SSuit   3.0
4                 115%   4.0
8                 130%   5.0
11                 QCS   6.0
0                    0   NaN
1                   00   NaN
2   100% Final Bracket   NaN
5              1192253   NaN
6          1192526 GLA   NaN
7            1192526 P   NaN

Check vial pos of
-------
0     0
1    00
Name: Name, dtype: object
                  Name count unique   top freq
0                    0     2      1  R:A2    2

#### Calibration Top

In [ ]:
'''
Download pytesseract installer from https://github.com/UB-Mannheim/tesseract/wiki and add to path
'''
# ! pip install pytesseract pillow
import pytesseract
pytesseract.pytesseract.tesseract_cmd= r'C:\Users\hpham\Downloads\tesseract.exe' #**

In [ ]:
# ! pip install easyocr
import fitz
import easyocr
from PIL import Image
import gc

# extract searched words or groups of words containing the searched words
def xtttii(page, word_h= word_h, word_v= word_v, reso= reso): 
    
    # Create empty frame for text at vertical and horizontal dimension and also the length of the specified text
    text_hf, text_vf= [],[]
    ocrlen_hf, ocrlen_vf= [], []
    outtext_hf= []

    # Looping through each resolution setting
    for r in reso:
    
        # Convert pdf to images
        mat= fitz.Matrix(r,r)
        pix= page.get_pixmap(matrix=mat)
        img= Image.frombytes('RGB',[pix.width,pix.height], pix.samples) #RGB
    
        # From image extract text horizontal   
        text= pytesseract.image_to_string(img)
        text= text.splitlines()
        text_h= [i for i in text if any(j in i for j in word_h)] 
        
        # From rotated image extract text vertical       
        rot=img.rotate(-90, expand= True) #*   
        result= easyocr.Reader(['en']).readtext(np.array(rot))
        ba=[]
        for a,b,c in result: 
            ba.append(b.split())
        text1= [i for j in ba for i in j] 
        text_v= [i for i in text1 if any(j in i for j in word_v)] 
        if len(text_v)== 0:
            ocrlen_v= 0
        else:
            ocrlen_v= len(text_v[0])

        # Clean to lighten memory usage
        del pix
        gc.collect()
        
        # Append to the original empty frame    
        text_hf.append(text_h)
        text_vf.append(text_v), ocrlen_vf.append(ocrlen_v)


    # Combining all resolution settings, create a frame containing all texts readouts
    for i in range(0,len(reso)):
        otext_hf= len(text_hf[i])
        outtext_hf.append(otext_hf)

        # Create another frame containing the lengths of each text block readouts
        text_hfj=[]
        for j in range(0,max(outtext_hf)):
            try:
                text_hfj.append(len(text_hf[i][j]))
            except:
                pass #
        ocrlen_hf.append(text_hfj)


    # Find the longest strings possible and its index for each block extracted
    ind_bh= (pd.DataFrame(ocrlen_hf).
            apply(lambda c: c[c==c.max()].index[-1])
    )
    besth=[]
    for i in range(0, len(ind_bh)):
        besth.append(text_hf[ind_bh[i]][i])
    
    bestvi= ocrlen_vf.index(max(ocrlen_vf))
    bestv= text_vf[bestvi]

    # Output the best horizontal and vertical text blocks that contains the searched words
    return besth, bestv
    

# Looping through all pages and output the blocks for each page
doc= fitz.open(file)
d=[]
for i in calib_page: 
    page= doc[i] 
    d.append(xtttii(page, word_h= word_h, word_v= word_v, reso= reso))
    del page
    gc.collect()
print(d)
doc.close() #1min for one

c:\Users\hpham\AppData\Local\miniconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[(['Calibration Type Lin, WithOffset, Avg Offset (CO) 0.1121', '[Evaluation Type ‘Area Slope (C1) 0.1657', '[Number of disabled Calibration Points 0 R-Square 0.999577', '1900. Atginine HCI Tntemal Arginine HCI IS" MS Quantitation', 'Amount', '6 CC‘“‘z”HISMG UT 457.8010 9.6887 | 2,456,596 | 20,906,449 _|'], ['Area']), (['Calibration Type | Lin, WithOffset, Avg Offset (CO) 0.0478', 'Evaluation Type | Area Slope (C1) 0.2268', 'Number of disabled Calibration Points 0 R-Square 0.999834', '500 Glutamine Internal ‘Glutamine IS" MS Quantitation', '“Amount'], ['Area']), (['Calibration Type Lin, WithOffset, Avg Offset (CO) 0.1297', 'Evaluation Type Area Slope (C1) 0.2075', 'Number of disabled Calibration Points ie) R-Square 0.999833', "500-5 Lysine HCl Internal ‘Lysine HCI IS' MS Quantitation", 'Amount'], ['Area']), (['Calibration Type | Lin, WithOffset, Avg Offset (CO) 0.0387', 'Evaluation Type | Area Slope (C1) 0.4188', '|Number of disabled Calibration Points 0 R-Square 0.999816', '460. Prolin

In [ ]:
# Confirm if the searched words are actually in the image
def cal_img(gh, text):
    if len(text[1])!=0:
        ya= gh[2] in text[1][0]
        if len(gh[0][0])>1:
            an = gh[0][0][i] in text[0][3]
        else:
            an = gh[0][i] in text[0][3]
        xa= gh[1] in text[0][4] 
    else:
        ya= 'no y-label'
        an = gh[0][i] in text[0][3]
        xa= 'no x-label'
    return an, xa, ya

for i in range(0,len(calib_page)):
    print(cal_img(gh, d[i]))


(True, True, True)
(True, True, True)
(True, True, True)
(True, True, True)


In [8]:
def regparams_pix(text, regp= 'Lin, WithOffset, Avg'):
    set= ('Lin, '+text[0][text[0].find('Lin') + len('Lin') + 2:][0:15]) == regp
    yint= float(text[0][-7:])
    slope= float(text[1][-6:])
    mse= float(text[2][-4:])
    return set, yint, slope, mse

set, yint, slope, mse = [],[],[],[]
for i in range(0,len(calib_page)):
    set0, yint0, slope0, mse0 = regparams_pix(d[i][0])
    set.append(set0), yint.append(yint0), slope.append(slope0), mse.append(mse0)
    
set, yint, slope, mse # ([True, True], [-0.0033, 0.0047], [0.2093, 0.2986], [4.0, 4.0])

([True, True, True, True],
 [0.1121, 0.0478, 0.1297, 0.0387],
 [0.1657, 0.2268, 0.2075, 0.4188],
 [9577.0, 9834.0, 9833.0, 9816.0])

#### Sum

In [9]:
def cleanframe_sum(df):
    for i in range(len(df)):
        try:
            f=df[i]
            
            f.columns=['No','Name1','RT','RTIS','Area','AreaIS','Ratio','Amount']
           
            f= f.reset_index(drop=True)
            f= f[~f.astype(str).apply(lambda row: row.str.contains('discard|rinse|old', case=False, na=False)).any(axis=1)]####
            f= f.set_index("No")

            # f= f[1:].iloc[:,].map(lambda cell: str(cell).replace(",",""))
            # for c in f.columns[1:]:
            #     f[c]= pd.to_numeric(f[c], errors='coerce')

            df[i]=f

        except:
            pass
        
        df.append(f)
    
    return df

In [10]:
def cleanframe_sum_1(df, col=['RT','RTIS','Area','AreaIS','Ratio']):
    df1=[]
    for f in df:
        f= f[1:].iloc[:,:-1].map(lambda cell: str(cell).replace(",",""))
        for c in col:
            f[c]= pd.to_numeric(f[c], errors='coerce')
        df1.append(f)       

    return df1

In [11]:
def xttcst(file,compage):
    with pdfplumber.open(file) as pdf:
        df0=[]
        for i in compage: 
            p= pdf.pages[i].extract_tables() 
            for x in p:
                df00= pd.DataFrame(x)
                df0.append(df00)
    df= pd.concat(df0, ignore_index=True)
    df=df.iloc[:,:8]
    
    sum_i= df[df.iloc[:,1].astype(str).str.contains('injection', case=False)].index
    
    if (len(sum_i) + len(analyte))==2:
        df1_0=df.iloc[sum_i[0]:] 
        return cleanframe_sum([df1_0])
    elif (len(sum_i) + len(analyte))==3:#len(sum_i)==2 & len(analyte)==1:
        df1_0=df.iloc[sum_i[0]:sum_i[1]]
        return cleanframe_sum([df1_0])
            
    elif (len(sum_i) + len(analyte))==4:#6,7 A 7,8 L 8,9 P 9
        df1_0=df.iloc[sum_i[0]:sum_i[1]]
        df1_1=df.iloc[sum_i[1]:]
        return cleanframe_sum([df1_0, df1_1])
    elif (len(sum_i) + len(analyte))==5:
        df1_0=df.iloc[sum_i[0]:sum_i[1]]
        df1_1=df.iloc[sum_i[1]:sum_i[2]]
        return cleanframe_sum([df1_0, df1_1])
        
    elif (len(sum_i) + len(analyte))==6:
        df1_0=df.iloc[sum_i[0]:sum_i[1]]
        df1_1=df.iloc[sum_i[1]:sum_i[2]]
        df1_2=df.iloc[sum_i[2]:]
        return cleanframe_sum([df1_0, df1_1, df1_2])
    elif (len(sum_i) + len(analyte))==7:
        df1_0=df.iloc[sum_i[0]:sum_i[1]]
        df1_1=df.iloc[sum_i[1]:sum_i[2]]
        df1_2=df.iloc[sum_i[2]:sum_i[3]]
        return cleanframe_sum([df1_0, df1_1, df1_2])
        
    else:
        df1_0=df.iloc[sum_i[0]:sum_i[1]]
        df1_1=df.iloc[sum_i[1]:sum_i[2]]
        df1_2=df.iloc[sum_i[2]:sum_i[3]]
        df1_3=df.iloc[sum_i[3]:]
        return cleanframe_sum([df1_0, df1_1, df1_2, df1_3])
    
if len(analyte)==1:
    df1_0= xttcst(file,compage)[0] 
elif len(analyte)==2:
    df1_0, df1_1 = xttcst(file,compage)[0], xttcst(file,compage)[1]
elif len(analyte)==3:
    df1_0, df1_1, df1_2 = xttcst(file,compage)[0], xttcst(file,compage)[1], xttcst(file,compage)[2]
else:
    df1_0, df1_1, df1_2, df1_3 = xttcst(file,compage)[0], xttcst(file,compage)[1], xttcst(file,compage)[2], xttcst(file,compage)[3]

In [12]:
def head_c(df):
    df1={}
    for t in header:
        count= df.apply(lambda col: col.str.contains(t, case=False).sum())
        df1[t]= count
    df1 = pd.DataFrame(df1)[2:]
    return df1

In [13]:
header.extend(list(analyte))

print(head_c(df1_0))
try:
    print(head_c(df1_1))
    print(head_c(df1_2))
    print(head_c(df1_3))
except NameError:
    pass

        IS  Roc  Ace  Arg  Glu  Lys  Pro
RTIS     1    0    0    1    0    0    0
Area     0    0    0    1    0    0    0
AreaIS   1    0    0    1    0    0    0
Ratio    0    0    0    1    0    0    0
Amount   0    0    0    1    0    0    0
        IS  Roc  Ace  Arg  Glu  Lys  Pro
RTIS     1    0    0    0    1    0    0
Area     0    0    0    0    1    0    0
AreaIS   1    0    0    0    1    0    0
Ratio    0    0    0    0    1    0    0
Amount   0    0    0    0    1    0    0
        IS  Roc  Ace  Arg  Glu  Lys  Pro
RTIS     1    0    0    0    0    1    0
Area     0    0    0    0    0    1    0
AreaIS   1    0    0    0    0    1    0
Ratio    0    0    0    0    0    1    0
Amount   0    0    0    0    0    1    0
        IS  Roc  Ace  Arg  Glu  Lys  Pro
RTIS     1    0    0    0    0    0    1
Area     0    0    0    0    0    0    1
AreaIS   1    0    0    0    0    0    1
Ratio    0    0    0    0    0    0    1
Amount   0    0    0    0    0    0    1


In [14]:
df= [df1_0, df1_1, df1_2, df1_3]
if len(analyte)==1:
    df2_0= cleanframe_sum_1(df)[0] 
elif len(analyte)==2:
    df2_0, df2_1 = cleanframe_sum_1(df)[0], cleanframe_sum_1(df)[1]
elif len(analyte)==3:
    df2_0, df2_1, df2_2 = cleanframe_sum_1(df)[0], cleanframe_sum_1(df)[1], cleanframe_sum_1(df)[2]
else:
    df2_0, df2_1, df2_2, df2_3 = cleanframe_sum_1(df)[0], cleanframe_sum_1(df)[1], cleanframe_sum_1(df)[2], cleanframe_sum_1(df)[3]

In [11]:
# df2_3.head()

In [15]:
def drt_c(df4_0):
    ### output the one that are not
    df4r_0= df4_0[~df4_0["Name1"].str.contains('0|00', case=False, na=False)].copy()
    df4r_0["dRT"]= (df4r_0["RT"]-df4r_0["RTIS"]).abs()
    df5_0= df4r_0[df4r_0["dRT"]>0.2]
    df5_0 
    if len(df5_0)!=0:
        print("Is it RocBro?")
    else:
        pass

In [16]:
drt_c(df2_0), drt_c(df2_1), drt_c(df2_2), drt_c(df2_3)

(None, None, None, None)

In [14]:
### RT shift
### RSD IS

In [17]:
df4na=df2_0[df2_0.isna().any(axis=1)]
print(df4na)
try:
    print(df2_1[df2_1.isna().any(axis=1)])
    print(df2_2[df2_2.isna().any(axis=1)])
    print(df2_3[df2_3.isna().any(axis=1)])
except NameError:
    pass

        Name1  RT  RTIS  Area    AreaIS  Ratio
No                                            
              NaN   NaN   NaN       NaN    NaN
              NaN   NaN   NaN       NaN    NaN
11         00 NaN   NaN   NaN       NaN    NaN
12          0 NaN  1.18   NaN  323792.0    NaN
43          0 NaN  1.18   NaN  359978.0    NaN
44         00 NaN   NaN   NaN       NaN    NaN
63    1192253 NaN  1.18   NaN  343119.0    NaN
64    1192253 NaN  1.18   NaN  343623.0    NaN
65    1192253 NaN  1.18   NaN  341727.0    NaN
66    1192253 NaN  1.18   NaN  339924.0    NaN
67    1192253 NaN  1.18   NaN  349346.0    NaN
68  1192253-6 NaN  1.18   NaN  339946.0    NaN
   Name1  RT  RTIS  Area    AreaIS  Ratio
No                                       
         NaN   NaN   NaN       NaN    NaN
         NaN   NaN   NaN       NaN    NaN
11    00 NaN   NaN   NaN       NaN    NaN
12     0 NaN  1.75   NaN  209422.0    NaN
43     0 NaN  1.75   NaN  229540.0    NaN
44    00 NaN   NaN   NaN       NaN    NaN
   Nam

In [18]:
def rnrsd(df4_0):
    df4_0["Name"]= df4_0['Name1'].str.split('-', n=1).str[0]
    df6_0=df4_0.groupby(["Name"])["Ratio"].mean()
    df6_0= pd.DataFrame(df6_0).reset_index().sort_values(by='Ratio')
    print(f'Analyte/IS ratio\n-------\n{df6_0}')
    df7_0= (df4_0.groupby(["Name"])["Ratio"].std()/df4_0.groupby(["Name"])["Ratio"].mean()*100).sort_values()
    print(f'RSD of Analyte/IS ratio\n-------\n{df7_0}\n\n\n')
    
    return df7_0, df6_0  

df7_0, df6_0= rnrsd(df2_0)
try:
    df7_1, df6_1= rnrsd(df2_1)
    df7_2, df6_2= rnrsd(df2_2)
    df7_3, df6_3= rnrsd(df2_3)
except NameError:
    pass

Analyte/IS ratio
-------
                  Name      Ratio
8            1192526 P   0.860850
10                 70%   5.935833
11                 85%   7.172650
3   100% Final Bracket   8.402000
4           100% SSuit   8.441250
7          1192526 GLA   8.458183
12                 QCS   8.483950
5                 115%   9.752300
9                 130%  10.890783
0                             NaN
1                    0        NaN
2                   00        NaN
6              1192253        NaN
RSD of Analyte/IS ratio
-------
Name
70%                   0.867028
QCS                   0.869355
1192526 GLA           0.996789
100% Final Bracket    1.010731
100% SSuit            1.233204
115%                  1.256323
85%                   1.314784
130%                  1.387613
1192526 P             1.791534
                           NaN
0                          NaN
00                         NaN
1192253                    NaN
Name: Ratio, dtype: float64



Analyte/IS ratio
-------
   

In [17]:
# df4_0= df4[df4['Name'].str.contains('100',case=False,na=False)]

# df9=[]
# calc1= QCS_conc * Volinit_qcs* Volser_qcs / Volfin_qcs
# for i in range(0,len(Volinit_std)):
#     calc= Std_conc * Volinit_std[i] * Volser_std[i] / Volfin_std[i]
#     df9.append(calc)
# df9.append(calc1)   
# df9

# sam_1= df6[df6['Name'].str.contains('|'.join(samcon))]
# df1_0['Dilution']=pd.to_numeric(df1_0['Dilution'], errors='coerce')
# dfad= df1_0.groupby('Name')['Dilution'].mean()
# df11= sam_0.merge(dfad, on='Name', how= 'left').merge(sam_1, on='Name', how= 'left')
# df11["Calc_amount"]= df11['Ratio'] * df9[2] * df11['Dilution'] * df11['Fill'] / df4_0['Ratio'].mean() ###
# df11['PC']=(df11["Calc_amount"]/df11['Claim']*100).astype(float).round(2)
# df11

NameError: name 'df4' is not defined

In [ ]:
# df6.iloc[0,1]/df6[df6['Name']=="130%"].iat[0,1] >0.01 ###

In [19]:
def regeq(QCS_conc, Std_conc, df4_0):
    from scipy.stats import linregress

    df9_0=[]
    calc1= QCS_conc * Volinit_qcs* Volser_qcs / Volfin_qcs
    for i in range(0,len(Volinit_std)):
        calc= Std_conc * Volinit_std[i] * Volser_std[i] / Volfin_std[i]
        df9_0.append(calc)
    df9_0.append(calc1)    

    call=['70%','85%','115%','130%','100%']
    pattern='|'.join(call)
    df4l_0=df4_0[df4_0["Name1"].str.contains(pattern, case=False, na=False) & 
            ~df4_0["Name1"].str.contains('bracket', case=False, na=False)] #.iloc[:30,:]
    # df4ll= pd.merge(levcal['Name1'],df4l, on='Name1', how= 'left').drop_duplicates(subset="Area")
    arry= df4l_0["Ratio"]#[:-1]

    arrx=np.repeat(np.array([df9_0[:5]]), injrep_cal[:5]) 

    slope_c, int, r, p, stderr = linregress(arrx.tolist(), arry.tolist())
    return slope_c, int, r, df9_0, arrx, arry
    
slope_c_0, int_0, r_0, df9_0, arrx_0, arry_0= regeq(QCS_conc[0], Std_conc[0], df2_0)
print(f'{analyte[0]}\n slope: {slope_c_0} \ny-intercept: {int_0} \nR^2: {r_0**2}\n')
try:
    slope_c_1, int_1, r_1, df9_1, arrx_1, arry_1= regeq(QCS_conc[1], Std_conc[1], df2_1)
    print(f'{analyte[1]}\n slope: {slope_c_1} \ny-intercept: {int_1} \nR^2: {r_1**2}\n')
    slope_c_2, int_2, r_2, df9_2, arrx_2, arry_2= regeq(QCS_conc[2], Std_conc[2], df2_2)
    print(f'{analyte[2]}\n slope: {slope_c_2} \ny-intercept: {int_2} \nR^2: {r_2**2}\n')
    slope_c_3, int_3, r_3, df9_3, arrx_3, arry_3= regeq(QCS_conc[3], Std_conc[3], df2_3)
    print(f'{analyte[3]}\n slope: {slope_c_3} \ny-intercept: {int_3} \nR^2: {r_3**2}\n')
except NameError:
    pass

Arg
 slope: 0.16566006973625022 
y-intercept: 0.11219666666665873 
R^2: 0.9963724416533092

Glu
 slope: 0.22675753448339936 
y-intercept: 0.047834444444444735 
R^2: 0.9973580565206726

Lys
 slope: 0.20746954050077845 
y-intercept: 0.1297555555555574 
R^2: 0.9976432386991739

Pro
 slope: 0.41883008601018074 
y-intercept: 0.038668888888888864 
R^2: 0.9974884888083497



In [19]:
i= 0 #**
abs((int-yint[i]))/yint[i] * 100 < .5, abs((slope_c-slope[i]))/slope[i] * 100 < 0.5, abs((r**2-mse[i]))/mse[i] * 100 < 0.7

NameError: name 'yint' is not defined

In [20]:
def sstrev(df6_0, int_0, slope_c_0, df9_0, dfo1_0= dfo1_0):
    df6_0["Theo_conc"]= (df6_0["Ratio"]- int_0)/slope_c_0
    df9_0=pd.DataFrame(df9_0)
    alev= dfo1_0[dfo1_0['Name'].str.contains('100', case=False, na=False) & ~dfo1_0['Name'].str.contains('bracket', case=False, na=False)]['Name'].iloc[0]
    if dfo1_0['Name'].str.contains('Check|qcs', case=False, na=False).any()==True:
        qcsorcs= dfo1_0[dfo1_0['Name'].str.contains('Check|qcs', case=False, na=False)]['Name'].iloc[0]
        df9_0["Conc"]=['70%','85%',alev,'115%','130%', qcsorcs]
    else:
        df9_0["Conc"]=['70%','85%',alev,'115%','130%','na']
    df9_0.columns=['Exp_amount','Name']
    df9a_0= pd.DataFrame({"Name":['100% Bracket1','100% Final Bracket'], #**
                        "Exp_amount": [df9_0.iloc[2,0], df9_0.iloc[2,0]]}) #**
    df9_0=pd.concat([df9_0,df9a_0],axis=0,ignore_index=True)
    df10_0= pd.merge(df6_0,df9_0, on='Name', how= 'right')
    df10_0["%_recovery"]= df10_0['Theo_conc']/df10_0['Exp_amount']*100 
    return df10_0

sstrev(df6_0, int_0, slope_c_0, df9_0, dfo1_0)

,Name,Ratio,Theo_conc,Exp_amount,%_recovery
0,70%,5.935833,35.154136,35.183232,99.917302
1,85%,7.172650,42.620128,42.722496,99.760388
2,100% SSuit,8.441250,50.277978,50.261760,100.032267
3,115%,9.752300,58.192076,57.801024,100.676548
4,130%,10.890783,65.064482,65.340288,99.577893
5,QCS,8.483950,50.535735,50.404570,100.260224
6,100% Bracket1,NaN,NaN,50.261760,NaN
7,100% Final Bracket,8.402000,50.041047,50.261760,99.560873


In [21]:
try:
    display(sstrev(df6_1, int_1, slope_c_1, df9_1, dfo1_0))
    display(sstrev(df6_2, int_2, slope_c_2, df9_2, dfo1_0))
    display(sstrev(df6_3, int_3, slope_c_3, df9_3, dfo1_0))
except:
    pass

,Name,Ratio,Theo_conc,Exp_amount,%_recovery
0,70%,2.451617,10.600672,10.590426,100.096747
1,85%,2.960550,12.845066,12.859803,99.885405
2,100% SSuit,3.487083,15.167077,15.129180,100.250487
3,115%,3.976633,17.325991,17.398557,99.582917
4,130%,4.516567,19.707095,19.667934,100.199110
5,QCS,3.566267,15.516275,15.690000,98.892765
6,100% Bracket1,NaN,NaN,15.129180,NaN
7,100% Final Bracket,3.459517,15.045508,15.129180,99.446947


,Name,Ratio,Theo_conc,Exp_amount,%_recovery
0,70%,2.385167,10.871047,10.890600,99.820459
1,85%,2.876683,13.240150,13.224300,100.119853
2,100% SSuit,3.353000,15.535989,15.558000,99.858521
3,115%,3.857233,17.966386,17.891700,100.417432
4,130%,4.315750,20.176429,20.225400,99.757874
5,QCS,3.284367,15.205177,15.210775,99.963198
6,100% Bracket1,NaN,NaN,15.558000,NaN
7,100% Final Bracket,3.341500,15.480559,15.558000,99.502243


,Name,Ratio,Theo_conc,Exp_amount,%_recovery
0,70%,0.777033,1.762921,1.772400,99.465201
1,85%,0.944750,2.163362,2.152200,100.518634
2,100% SSuit,1.099233,2.532207,2.532000,100.008172
3,115%,1.259900,2.915815,2.911800,100.137892
4,130%,1.414817,3.285695,3.291600,99.820594
5,QCS,1.109733,2.557277,2.572875,99.393743
6,100% Bracket1,NaN,NaN,2.532000,NaN
7,100% Final Bracket,1.087683,2.504630,2.532000,98.919041


In [22]:
def samrev(df6_0, an, sam_0= sam_0, dfo1_0= dfo1_0):
    sam_1= df6_0[df6_0['Name'].str.contains('|'.join(samcon))].drop(columns='Ratio') 

    dfo1_0['Dilution']=pd.to_numeric(dfo1_0['Dilution'], errors='coerce')
    dfad= dfo1_0.groupby('Name')['Dilution'].mean()
    df11_0= sam_0.merge(dfad, on='Name', how= 'left').merge(sam_1, on='Name', how='left' )
    df11_0["Calc_amount"]= df11_0['Theo_conc'] * df11_0['Dilution'] * df11_0['Fill']
    df11_0['PC']=(df11_0["Calc_amount"]/df11_0[an]*100).astype(float).round(2)
    return df11_0[['Name','Fill',an, 'Dilution','Theo_conc', 'Calc_amount','PC']]

samrev(df6_0, 'Claim0')

,Name,Fill,Claim0,Dilution,Theo_conc,Calc_amount,PC
0,1192526 GLA,1,50,1.0,50.380195,50.380195,100.76
1,1192526 P,1,-15,10.0,4.519214,45.192142,-301.28
2,1192235,1,-15,NaN,NaN,NaN,NaN


In [23]:
try:
    display(samrev(df6_1, 'Claim1'))
    display(samrev(df6_2, 'Claim2'))
    display(samrev(df6_3, 'Claim3'))
except:
    pass

,Name,Fill,Claim1,Dilution,Theo_conc,Calc_amount,PC
0,1192526 GLA,1,15,1.0,14.865506,14.865506,99.10
1,1192526 P,1,-15,10.0,1.303884,13.038842,-86.93
2,1192235,1,25,NaN,NaN,NaN,NaN


,Name,Fill,Claim2,Dilution,Theo_conc,Calc_amount,PC
0,1192526 GLA,1,15,1.0,15.343511,15.343511,102.29
1,1192526 P,1,-50,10.0,1.012813,10.128127,-20.26
2,1192235,1,-50,NaN,NaN,NaN,NaN


,Name,Fill,Claim3,Dilution,Theo_conc,Calc_amount,PC
0,1192526 GLA,1,-1,1.0,25.780099,25.780099,-2578.01
1,1192526 P,1,25,10.0,2.531570,25.315702,101.26
2,1192235,1,-25,NaN,NaN,NaN,NaN


#### Calib Bot D

In [24]:
with pdfplumber.open(file) as pdf:
    df0=[]
    for p in calib_page1:
        p= pdf.pages[p].extract_tables()
        for t in p:
            df00= pd.DataFrame(t)
            df0.append(df00)
    df= pd.concat(df0, ignore_index=True)
        
dfc= pd.DataFrame(df)
cal_i= dfc[dfc.astype(str).apply(lambda row:row.str.contains('injection', case=False,na=False),axis=1).any(axis=1)].index


In [25]:
def cleanframe_cal(dfc1_0):
    dfc1_0.columns= dfc1_0.iloc[0].to_numpy().flatten()
    dfc1_0= dfc1_0[['Injection Name','X Value','Y Value']].copy()
    for c in ['X Value','Y Value']:
        dfc1_0[c]= pd.to_numeric(dfc1_0[c], errors='coerce')
    dfc1_0.columns= dfc1_0.columns.str.lower()
    dfc1_0= dfc1_0.dropna()
    
    return dfc1_0

dfc1_0= dfc.iloc[cal_i[0]:cal_i[1]].reset_index(drop=True)
dfc1_0= cleanframe_cal(dfc1_0)

In [26]:
try:
    dfc1_1= cleanframe_cal(dfc.iloc[cal_i[1]:cal_i[2]].reset_index(drop=True))
    dfc1_2= cleanframe_cal(dfc.iloc[cal_i[2]:cal_i[3]].reset_index(drop=True))
    dfc1_3= cleanframe_cal(dfc.iloc[cal_i[3]:].reset_index(drop=True))
except:
    pass

In [27]:
try:
    display(np.allclose(np.array(dfc1_0['x value']),np.array(pd.to_numeric(arrx_0, errors='coerce')), rtol=0.0001))
    display(np.allclose(np.array(dfc1_0['y value']),np.array(pd.to_numeric(arry_0, errors='coerce')), rtol=0.0001))
except ValueError:
    print('frames >< size')

True

True

#### MS

In [28]:
file= 'lc.xlsx'
# df=pd.read_csv('ms1.csv') ### need saved in utf-8
df0= pd.read_excel(file, sheet_name= 'msinst')
df= pd.read_excel(file, sheet_name= 'msms').dropna(axis=1)


#**
sheet= 'msq'

### check levels as well for when no calib curve used

In [29]:
df['Compound']= df.iloc[:,0].str.replace(' HCl','')

for i in df.columns:
    if df[i].astype(str).str.contains('Autodetect', case=False, na=False).all():
        df.drop(columns=i, inplace= True)
    elif df[i].astype(str).str.contains('MS_Quantitation', case=False, na=False).all():
        df.drop(columns=i, inplace= True)
    elif df[i].astype(str).str.contains('Area', case=False, na=False).all():
        df.drop(columns=i, inplace= True)
    elif df[i].astype(str).str.contains(r'^[a-zA-Z0-9]{1,2}$', regex=True, case=False, na=False).all():
        df.drop(columns=i, inplace= True)

df=df.select_dtypes(exclude='number').drop(df.columns[0], axis=1)

df= df[['Compound'] + [col for col in df.columns if col != 'Compound']]

for i in df.columns:
    if df[i].astype(str).str.contains('Lin,', case=False, na=False).all():
        df.rename(columns={i:'Calib_setting'}, inplace= True)
    elif df[i].astype(str).str.contains('istd', case=False, na=False).any():
        df.rename(columns={i:'IS_setting'}, inplace= True)
    elif df[i].astype(str).str.contains('ESI SRM', case=False, na=False).any():
        df.rename(columns={i:'MSMS'}, inplace= True)

print(f'For reference')        
df

For reference


,Compound,Unnamed: 2,MSMS,Unnamed: 4,Unnamed: 5,IS_setting,Calib_setting
0,Lysine IS,0.500 AG,+ c ESI SRM ms2 155.100 [92.199-92.201],"SI:0, RSI:0",155.10 / 92.20,ISTD Internal,"Lin, WithOffset, Avg"
1,Lysine,1.000 AF,+ c ESI SRM ms2 147.100 [84.199-84.201],"SI:0, RSI:0",147.10 / 84.20,Internal Lysine HCl IS (MS_Quantitation),"Lin, WithOffset, Avg"
2,Arginine IS,0.500 AG,+ c ESI SRM ms2 182.200 [77.199-77.201],"SI:0, RSI:0",182.20 / 77.20,ISTD Internal,"Lin, WithOffset, Avg"
3,Arginine,0.500 AG,+ c ESI SRM ms2 175.200 [70.199-70.201],"SI:0, RSI:0",175.20 / 70.20,Internal Arginine HCl IS (MS_Quantitation),"Lin, WithOffset, Avg"
4,Glutamine IS,0.500 AG,+ c ESI SRM ms2 152.100 [89.199-89.201],"SI:0, RSI:0",152.10 / 89.20,ISTD Internal,"Lin, WithOffset, Avg"
5,Glutamine,0.500 AG,+ c ESI SRM ms2 147.100 [84.199-84.201],"SI:0, RSI:0",147.10 / 84.20,Internal Glutamine IS (MS_Quantitation),"Lin, WithOffset, Avg"
6,Proline IS,0.500 AG,+ c ESI SRM ms2 119.100 [73.099-73.101],"SI:0, RSI:0",119.10 / 73.10,ISTD Internal,"Lin, WithOffset, Avg"
7,Proline,0.500 AG,+ c ESI SRM ms2 116.100 [70.099-70.101],"SI:0, RSI:0",116.10 / 70.10,Internal Proline IS (MS_Quantitation),"Lin, WithOffset, Avg"


In [30]:
df[['b','a']]=df['MSMS'].str.split('[',expand=True)
df['Precursor (m/z)']= pd.to_numeric(df['b'].str[15:]).round(1)
df['Product (m/z)']= pd.to_numeric(df['a'].str.split('-').str[0]).map(lambda i: f"{i:.1f}") #normal round dont do 2 decplc
df['Mode']= df['b'].str[:15]

In [31]:
print(f'{df[df['IS_setting'].astype(str).str.contains('IStd', case=False, na=False)]['Compound'].str.contains('IS', case=False, na=False).all() } IS are labelled as IS')
try:
    if injrep_cal[0] != 0:
        print(f'{df[~df[df['Calib_setting']=='Lin, WithOffset, Avg']['Compound'].str.contains('IS', case=False, na=False)]['Compound'].shape[0] == df.shape[0]/2} Analytes have 3calib settings')
except KeyError:
    print('No Calib done')
print(f'{df['Mode'].str.contains('ESI SRM ms2', case=False, na=False).all()} MS mode is ESI SRM ms2 ')

True IS are labelled as IS
True Analytes have 3calib settings
True MS mode is ESI SRM ms2 


In [32]:
msinst= df0.dropna()
msinst.columns=msinst.iloc[0]
try:
    msinst=msinst[['Compound','Precursor (m/z)','Product (m/z)','Collision Energy (V)','RF Lens (V)']]
except:
    pass
msinst= msinst[1:]
msinst

41,Compound,Precursor (m/z),Product (m/z),Collision Energy (V),RF Lens (V)
42,Proline,116.1,70.1,14,40
43,Proline IS,119.1,73.1,14,40
44,Glutamine,147.1,84.2,17,46
45,Lysine,147.1,84.2,17,46
46,Glutamine IS,152.1,89.2,17,46
47,Lysine IS,155.1,92.2,17,46
48,Arginine,175.2,70.2,23,52
49,Arginine IS,182.2,77.2,23,52


In [33]:
df1= df0[df0.astype(str).apply(lambda row: row.str.contains('nan', case=False, na=False)).any(axis=1)].dropna(axis=1)
df1[['Setting','Value']]= df1.iloc[:,0].str.split(':',expand=True)
df1=df1.drop(columns='Method Summary')

if 'Mode' in df.columns:
    if df['Mode'].str[:1].unique()[0] =='+':
        a= df1[df1['Setting']=='Positive Ion (V)']
    else:
        a= df1[df1['Setting']=='Negative Ion (V)']
else:
    a= pd.concat([df1[df1['Setting']=='Negative Ion (V)'],df1[df1['Setting']=='Positive Ion (V)']])
    
pd.concat([a, 
           df1[df1['Setting']=='Vaporizer Temp (°C)'], 
           df1[df1['Setting']=='Sheath Gas (Arb)'], df1[df1['Setting']=='Sweep Gas (Arb)'], df1[df1['Setting']=='Aux Gas (Arb)'],
           df1[df1['Setting']=='Ion Transfer Tube Temp (°C)'], df1[df1['Setting']=='CID Gas (mTorr)'], 
           df1[df1['Setting']=='Dwell Time (ms)'], df1[df1['Setting']=='Cycle Time (sec)'], 
           df1[df1['Setting']=='Q1 Resolution (FWHM)'], df1[df1['Setting']=='Q3 Resolution (FWHM)'],
           ])

,Setting,Value
6,Positive Ion (V),4000
13,Vaporizer Temp (°C),400
9,Sheath Gas (Arb),50
11,Sweep Gas (Arb),2
10,Aux Gas (Arb),15
12,Ion Transfer Tube Temp (°C),350
31,CID Gas (mTorr),1.5
27,Cycle Time (sec),1.2
29,Q1 Resolution (FWHM),0.7
30,Q3 Resolution (FWHM),0.7


In [34]:
df=df[['Compound','Precursor (m/z)','Product (m/z)']]

### better do a merge first then spot check

b= df.sort_values(by='Precursor (m/z)').reset_index(drop=True)
c= msinst.sort_values(by='Precursor (m/z)').reset_index(drop=True)[['Compound','Precursor (m/z)','Product (m/z)']]

c['Product (m/z)']= pd.to_numeric(c['Product (m/z)'], errors='coerce')
b['Product (m/z)']= pd.to_numeric(b['Product (m/z)'], errors='coerce')

if np.array_equal(b,c) != True:
    print(f'DB CLICK setting to CHECK- 2237 Pro?!\n\n {c[c!=b]} \n vs Instrument set \n {b[b!=c]}')
else:
    pass

DB CLICK setting to CHECK- 2237 Pro?!

 41   Compound Precursor (m/z)  Product (m/z)
0         NaN             NaN            NaN
1         NaN             NaN            NaN
2   Glutamine             NaN            NaN
3      Lysine             NaN            NaN
4         NaN             NaN            NaN
5         NaN             NaN            NaN
6         NaN             NaN            NaN
7         NaN             NaN            NaN 
 vs Instrument set 
     Compound  Precursor (m/z)  Product (m/z)
0        NaN              NaN            NaN
1        NaN              NaN            NaN
2     Lysine              NaN            NaN
3  Glutamine              NaN            NaN
4        NaN              NaN            NaN
5        NaN              NaN            NaN
6        NaN              NaN            NaN
7        NaN              NaN            NaN


#### LC

In [35]:
def cleanframe(frame, col):
    frame[col]= frame[col].astype(str)
    output=[]
    for i,r in frame.iterrows():
        if 'Wave' in r[col]:
            output.append('Detector')
        elif 'Bandwidth' in r[col]:
            output.append('Bandwidth')
        elif 'Temperature' in r[col] and 'col' in r[col].lower():
            output.append('Column')
        elif 'rid' in r['Command'].lower() and 'Temperature' in r['Command']:
            output.append('RID detector')
        elif 'Thermostat.N' in r['Command'] or 'Temperature.N' in r['Command'] and 'Sampler' in r[col]:
            output.append('Autosampler')
        elif 'Draw' in r[col]:
            output.append('Draw speed')
        elif 'Flow' in r[col]:
            output.append('Flow') 
        else:
            output.append('Run time')
    frame['Params']= output

In [36]:
def lcparams1(file,sheet):
    
    a= pd.concat([df[df['Command']=='ColumnComp.CC.Temperature.Nominal'], df[df['Command']=='ColumnOven.Temperature.Nominal'],
                  df[df['Command']=='SamplerModule.Temperature.Nominal'], df[df['Command']=='Sampler.Temperature.Nominal'], #df[df['Command']=='Sampler.TempCtrl'], 
                  df[df['Command']=='PumpModule.Pump.Flow.Nominal'], 
                  df[df['Command']=='SamplerModule.Sampler.DrawSpeed'], df[df['Command']=='Sampler.DrawSpeed'],
                  df[df['Command']=='UV.UV_VIS_1.Wavelength'], df[df['Command']=='UV.UV_VIS_2.Wavelength'], df[df['Command']=='UV.UV_VIS_3.Wavelength'], df[df['Command']=='UV.UV_VIS_4.Wavelength'],
                  df[df['Command']=='UV.UV_VIS_1.RefWavelength'],
                     df[df['Stage']=='Run'], 
                  ]).drop_duplicates(subset=["Command"])
                # |df.astype(str).apply(lambda row: row.str.contains('Duration',case=False,na=False)).any(axis=1)
    
    cleanframe(a, 'Command')
    
    head_i= df[df.astype(str).apply(lambda row: row.str.contains('duration', case=False, na=False)).any(axis=1)].index.values[0]
    dfg=df.iloc[head_i:,:].reset_index(drop=True)
    # dfg
    gd= dfg["Time"].unique()
    # gd
    # dfgd=[{'Time': "0", '%B':val1}]
    dfgd=[]
    refiy = 1
    for i in gd[1:]:
        refix = dfg[dfg["Time"]==i].index[0]
        dat= {
            'Time': f'{i}',
            '%B': dfg.iat[refix+2,refiy+2]
        }
        dfgd.append(dat)
    dfgd=pd.DataFrame(dfgd)

    return a[['Params','Value']], dfgd

In [37]:
def lcparams2(file,sheet):
        
    a= df[df['Command'].isin(["Agilent.COLCOMP1.TemperatureControlLeft.Nominal", 
                          "Agilent.SAMPLER1.Thermostat.Nominal", #"Agilent.SAMPLER1.Thermostat.ControlMode",
                          "Agilent.PUMP1.Flow.Nominal", 
                          "Agilent.DAD1.DAD1_Signal_A.Wavelength", "Agilent.DAD1.DAD1_Signal_B.Wavelength", 
                          "Agilent.DAD1.DAD1_Signal_A.Bandwidth", "Agilent.DAD1.DAD1_Signal_A.RefBandwidth", 
                          "Agilent.DAD1.DAD1_Signal_A.RefWavelength",
                          "Agilent.RID1.Temperature.Nominal",
                        ])
                        |
                        df.astype(str).apply(lambda row: row.str.contains('Duration',case=False,na=False)).any(axis=1)
            ].drop_duplicates(subset=["Command"])[['Command','Value']]

    # a['Setting']= a['Command'].str.contains(i, case= False, na=False)
    # a['Setting']= a['Setting'].map({True: l, False: ''})
    cleanframe(a, 'Command')
    
    refx = df[df["Stage"]=="Equilibration"].index[0]
    refy = df.columns.get_loc("Stage")
    val1= df.iat[refx+3,refy+3]

    head_i= df[df.astype(str).apply(lambda row: row.str.contains('duration', case=False, na=False)).any(axis=1)].index.values[0]
    dfg=df.iloc[head_i:,:].reset_index(drop=True)
    # dfg
    gd= dfg["Time"].unique()
    # gd
    dfgd=[{'Time': "0", '%B':val1}]
    # dfgd=[]
    refiy = 1
    for i in gd[2:]:
        refix = dfg[dfg["Time"]==i].index[0]
        dat= {
            'Time': f'{i}',
            '%B': dfg.iat[refix+1,refiy+2] #x+1 for 52
        }
        dfgd.append(dat)
    dfgd=pd.DataFrame(dfgd)

    return a[['Params','Value']], dfgd

In [38]:
def lcparams3(file,sheet):
    
    a= pd.concat([df[df['Command']=='ColumnComp.CC.Temperature.Nominal'], df[df['Command']=='ColumnOven.Temperature.Nominal'],
                  df[df['Command']=='SamplerModule.Temperature.Nominal'], df[df['Command']=='Sampler.Temperature.Nominal'], #df[df['Command']=='Sampler.TempCtrl'], 
                  df[df['Command']=='PumpModule.Pump.Flow.Nominal'], 
                  df[df['Command']=='SamplerModule.Sampler.DrawSpeed'],
                  df[df['Command']=='UV.UV_VIS_1.Wavelength'], df[df['Command']=='UV.UV_VIS_2.Wavelength'], df[df['Command']=='UV.UV_VIS_3.Wavelength'], df[df['Command']=='UV.UV_VIS_4.Wavelength'],
                  df[df['Stage']=='Run'], 
                  ]).drop_duplicates(subset=["Command"])
                # |df.astype(str).apply(lambda row: row.str.contains('Duration',case=False,na=False)).any(axis=1)
    
    cleanframe(a, 'Command')

    head_i= df[df.astype(str).apply(lambda row: row.str.contains('duration', case=False, na=False)).any(axis=1)].index.values[0]
    dfg=df.iloc[head_i:,:].reset_index(drop=True)
    
    gd= dfg["Time"].unique()
    dfg_0=dfg[dfg.astype(str).apply(lambda row: row.str.contains('%B.Value', regex=False, case=False, na=False)).any(axis=1)]['Value']

    dfgrd= pd.DataFrame([gd[1:],dfg_0]).T
    dfgrd.columns= ['Time','%B']
    dfgd=pd.concat([pd.DataFrame([{'Time': "0", '%B':0}]),dfgrd])

    return a[['Params','Value']], dfgd

In [39]:
dfi=pd.read_excel(file, sheet_name=sheet)
head_i= dfi[dfi.astype(str).apply(lambda row: row.str.contains('command', case=False, na=False)).any(axis=1)].index.values[0]
df=dfi.iloc[head_i:,:].reset_index(drop=True)
df.columns=df.iloc[0]

if df['Command'].astype(str).str.contains('agilen', case=False, na=False).any() != False: # '39','52','61.','37'
    a= lcparams2(file, sheet)
elif sheet == 'msq':
    a= lcparams3(file, sheet)
else: # '67','63','55','56','45.','ms','42','60'
    a= lcparams1(file, sheet)
a

(0       Params                    Value
 17      Column               25.00 [°C]
 40        Flow           0.900 [ml/min]
 9   Draw speed             5.000 [µl/s]
 36    Run time  Duration = 10.000 [min],
   Time        %B
 0    0         0
 0    0   0.0 [%]
 1  5.5   0.0 [%]
 2  5.6  60.0 [%]
 3  7.1  60.0 [%]
 4  7.2   0.0 [%]
 5   10   0.0 [%])